<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Pui/American_Call_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl https://colab.chainer.org/install |sh -
import cupy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  13057      0 --:--:-- --:--:-- --:--:-- 13057
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9 MB 31 kB/s 
     |████████████████████████████████| 1.0 MB 31.4 MB/s 
+ set +ex
Installation succeeded!


# Test (Skip this if not trying to test, to make sure that functions are defined correctly in cells below without running this cell)

In [ ]:
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

x_sub = jnp.array([[1,1,1,1,1],
                   [1.08,1.07,0.97,0.77,0.84],
                   [1.08**2,1.07**2,0.97**2,0.77**2,0.84**2]]).transpose()
y_sub = jnp.array([0, 0.07*0.94176, 0.18*0.94176, 0.2*0.94176, 0.09*0.94176])

print(jax.numpy.linalg.lstsq(x_sub, y_sub)[0])
print(x_sub.dot(jax.numpy.linalg.lstsq(x_sub, y_sub)[0]))

[-1.0699916  2.9834177 -1.8135799]
[0.03674006 0.04589796 0.11752641 0.1519686  0.15641725]


In [2]:
# call
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  #print(out)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  print('out_avg')
  print(out_avg)

  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  #print(ITM_matrix)
  CF_matrix = jnp.zeros((numpaths, numsteps)) # matrix that will store cash flows
  #print(CF_matrix)

  # update time T
  CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                   idx=jax.ops.index[:,-1], 
                                   y=jnp.maximum(out_avg[:,-1]-K, 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # print(jax.numpy.linalg.lstsq(out_avg[jnp.array(jnp.where(ITM_matrix[:,-2])[0]),-2], 
  #                              y_sub))
  # ITM_index = jnp.where(ITM_matrix[:,-3])[0] # ITM index for this timestep
  # print(out_avg[ITM_index,-3])
  # X = out_avg[ITM_index,-3]
  # print(CF_matrix[ITM_index,-3:]) # subset of CF matrix, ITM, from timestep i to end
  # print(jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_matrix[:,-3].sum(), axis=0)) # discount matrix, same structure as the subset of CF matrix above
  # print(jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # take exponential
  # print(CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # multiply CF and discount matrices
  # print((CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)) # Y
  # Y = (CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)
  # print(jnp.array([X**0,X**1,X**2]).transpose()) # matrix of [X**0, X**1, X**2]
  # X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
  # print(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # regression coefficients
  # print(X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])) # predicted Y - ECV - expected continue value
  # ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])
  # print(X-K)
  # print(X-K > ECV) # test if S-K > ECV
  # print((X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0

  print('ITM_matrix')
  print(ITM_matrix)
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    print(i)
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    print('X')
    print(X)
    Y = CF_matrix[ITM_index,-i:].dot(jnp.exp(-r[0]*jnp.arange(i) * dt)) # Y is computed by discounting future cashflows
    print('Y')
    print(Y)
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
    print('ECV')
    print(ECV)

    # update column i in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                     idx=jax.ops.index[jnp.array(ITM_index),-i],  # index subsetting ITM paths for each timestep
                                     y=(X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0
    
    print('CF_matrix')
    print(CF_matrix)
    print(jnp.where(X-K > ECV)[0])
    print(ITM_index[jnp.where(X-K > ECV)[0]])
    # let columns after the first non-zero number be 0 in each row in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                  idx=jax.ops.index[ITM_index[jnp.where(X-K > ECV)[0]], -(i-1):], # only for paths where X-K > ECV, update columns after to 0
                                  y=0)
    print('CF_matrix after removing nonzeros')
    print(CF_matrix)

  return  CF_matrix.dot(jnp.exp(-r[0]*jnp.arange(numsteps) * dt)).mean() # discount cash matrix and take mean for all paths to get price

numstocks = 3
numpaths = 5

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 6
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 100.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

out_avg
[[105.99862  100.0316   104.09863  111.01766  116.74928  109.565445]
 [ 90.274536  87.21661   96.076744  89.459885  91.10431   80.85022 ]
 [ 92.95862   96.04223   99.153305 104.86502  106.42198  104.403915]
 [ 93.90787   88.73891   91.71341   98.12489   96.75338   95.12549 ]
 [100.09762  112.7153   108.99028  119.56731  114.38698  103.237686]]
ITM_matrix
[[ True  True  True  True  True  True]
 [False False False False False False]
 [False False False  True  True  True]
 [False False False False False False]
 [ True  True  True  True  True  True]]
2
X
[116.74928 106.42198 114.38698]
Y
[9.549516  4.3965816 3.2322946]
ECV
[7.203638  3.633318  6.3367405]
CF_matrix
[[ 0.         0.         0.         0.        16.749283   9.565445 ]
 [ 0.         0.         0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.         6.421982   4.4039154]
 [ 0.         0.         0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.        1

In [ ]:
# call
# large numpaths & numsteps to test price here
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  CF_matrix = jnp.zeros((numpaths, numsteps)) # matrix that will store cash flows

  # update time T
  CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                   idx=jax.ops.index[:,-1], 
                                   y=jnp.maximum(out_avg[:,-1]-K, 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    Y = CF_matrix[ITM_index,-i:].dot(jnp.exp(-r[0]*jnp.arange(i) * dt)) # Y is computed by discounting future cashflows
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value

    # update column i in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                     idx=jax.ops.index[jnp.array(ITM_index),-i],  # index subsetting ITM paths for each timestep
                                     y=(X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0
    
    # let columns after the first non-zero number be 0 in each row in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                  idx=jax.ops.index[ITM_index[jnp.where(X-K > ECV)[0]], -(i-1):], # only for paths where X-K > ECV, update columns after to 0
                                  y=0)

  return  CF_matrix.dot(jnp.exp(-r[0]*jnp.arange(numsteps) * dt)).mean() # discount cash matrix and take mean for all paths to get price

numstocks = 1
numpaths = 1000

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 100
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([200.]*numstocks) # must be float

T = 2.0
K = 190.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r = drift, sigma_avg, T
# 100, 100, 0.01, 0.1443, 1
# option price should be around 6.3128
# test price: 6.3027

#S, K, r = drift, sigma, T
# 200, 190, 0.01, 0.1443, 2
# option price should be around 23.40856
# test price: 23.543825
####################################################################################

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

32.084904


In [ ]:
# call
# now change r to be an array
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  CF_matrix = jnp.zeros((numpaths, numsteps)) # matrix that will store cash flows

  # update time T
  CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                   idx=jax.ops.index[:,-1], 
                                   y=jnp.maximum(out_avg[:,-1]-K, 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    Y = CF_matrix[ITM_index,-i:].dot(jnp.exp(-(r[-i:].mean())*jnp.arange(i) * dt)) # Y is computed by discounting future cashflows
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value

    # update column i in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                     idx=jax.ops.index[jnp.array(ITM_index),-i],  # index subsetting ITM paths for each timestep
                                     y=(X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0
    
    # let columns after the first non-zero number be 0 in each row in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                  idx=jax.ops.index[ITM_index[jnp.where(X-K > ECV)[0]], -(i-1):], # only for paths where X-K > ECV, update columns after to 0
                                  y=0)

  return  CF_matrix.dot(jnp.exp(-(r.mean())*jnp.arange(numsteps) * dt)).mean() # discount cash matrix and take mean for all paths to get price

numstocks = 3
numpaths = 5

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 6
drift = jnp.array([0.01]*numstocks)
#r = drift # let r = drift to match B-S
r =jnp.array(np.random.random(numsteps) * 0.1)

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 100.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

8.339826
[0.19524175 0.25062665 0.22243044]


In [ ]:
# put
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  #print(out)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  print('out_avg')
  print(out_avg)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  #print(ITM_matrix)
  CF_matrix = jnp.zeros((numpaths, numsteps)) # matrix that will store cash flows
  #print(CF_matrix)

  # update time T
  CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                   idx=jax.ops.index[:,-1], 
                                   y=jnp.maximum(K-out_avg[:,-1], 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # print(jax.numpy.linalg.lstsq(out_avg[jnp.array(jnp.where(ITM_matrix[:,-2])[0]),-2], 
  #                              y_sub))
  # ITM_index = jnp.where(ITM_matrix[:,-3])[0] # ITM index for this timestep
  # print(out_avg[ITM_index,-3])
  # X = out_avg[ITM_index,-3]
  # print(CF_matrix[ITM_index,-3:]) # subset of CF matrix, ITM, from timestep i to end
  # print(jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_matrix[:,-3].sum(), axis=0)) # discount matrix, same structure as the subset of CF matrix above
  # print(jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # take exponential
  # print(CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # multiply CF and discount matrices
  # print((CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)) # Y
  # Y = (CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)
  # print(jnp.array([X**0,X**1,X**2]).transpose()) # matrix of [X**0, X**1, X**2]
  # X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
  # print(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # regression coefficients
  # print(X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])) # predicted Y - ECV - expected continue value
  # ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])
  # print(X-K)
  # print(X-K > ECV) # test if S-K > ECV
  # print((X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0

  print('ITM_matrix')
  print(ITM_matrix)
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    print(i)
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    print('X')
    print(X)
    Y = CF_matrix[ITM_index,-i:].dot(jnp.exp(-(r[-i:].mean())*jnp.arange(i) * dt)) # Y is computed by discounting future cashflows
    print('Y')
    print(Y)
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
    print('ECV')
    print(ECV)

    # update column i in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                     idx=jax.ops.index[jnp.array(ITM_index),-i],  # index subsetting ITM paths for each timestep
                                     y=(K-X > ECV).astype(int) * (K-X)) # X-K if X-K > ECV, else 0
    
    print('CF_matrix')
    print(CF_matrix)
    print(jnp.where(K-X > ECV)[0])
    print(ITM_index[jnp.where(K-X > ECV)[0]])
    # let columns after the first non-zero number be 0 in each row in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                  idx=jax.ops.index[ITM_index[jnp.where(K-X > ECV)[0]], -(i-1):], # only for paths where X-K > ECV, update columns after to 0
                                  y=0)
    print('CF_matrix after removing nonzeros')
    print(CF_matrix)

  return  CF_matrix.dot(jnp.exp(-(r.mean())*jnp.arange(numsteps) * dt)).mean() # discount cash matrix and take mean for all paths to get price

numstocks = 3
numpaths = 5

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 6
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 100.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

out_avg
[[105.99862  100.0316   104.09863  111.01766  116.74928  109.565445]
 [ 90.274536  87.21661   96.076744  89.459885  91.10431   80.85022 ]
 [ 92.95862   96.04223   99.153305 104.86502  106.42198  104.403915]
 [ 93.90787   88.73891   91.71341   98.12489   96.75338   95.12549 ]
 [100.09762  112.7153   108.99028  119.56731  114.38698  103.237686]]
ITM_matrix
[[False False False False False False]
 [ True  True  True  True  True  True]
 [ True  True  True False False False]
 [ True  True  True  True  True  True]
 [False False False False False False]]
2
X
[91.10431 96.75338]
Y
[19.11789   4.866394]
ECV
[19.118011   4.8664856]
CF_matrix
[[ 0.         0.         0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.         0.        19.14978  ]
 [ 0.         0.         0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.         0.         4.8745117]
 [ 0.         0.         0.         0.         0.         0.       ]]
[]
[]
C

In [6]:
# put
# large numpaths & numsteps to test price here
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_matrix = jnp.zeros((numpaths, numsteps)) # matrix that will store cash flows

  # update time T
  CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                   idx=jax.ops.index[:,-1], 
                                   y=jnp.maximum(K-out_avg[:,-1], 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    Y = CF_matrix[ITM_index,-i:].dot(jnp.exp(-(r[-i:].mean())*jnp.arange(i) * dt)) # Y is computed by discounting future cashflows

    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value

    # update column i in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                     idx=jax.ops.index[jnp.array(ITM_index),-i],  # index subsetting ITM paths for each timestep
                                     y=(K-X > ECV).astype(int) * (K-X)) # K-X if K-X > ECV, else 0
     
    # let columns after the first non-zero number be 0 in each row in CF_matrix
    CF_matrix = jax.ops.index_update(x=CF_matrix, 
                                  idx=jax.ops.index[ITM_index[jnp.where(K-X > ECV)[0]], -(i-1):], # only for paths where K-X > ECV, update columns after to 0
                                  y=0)

  return  CF_matrix.dot(jnp.exp(-(r.mean())*jnp.arange(numsteps) * dt)).mean() # discount cash matrix and take mean for all paths to get price

numstocks = 1
numpaths = 1000

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 100
drift = jnp.array([0.06]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)* 0.2 * 0.2
initial_stocks = jnp.array([40.]*numstocks) # must be float

T = 1.0
K = 40.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#################################################################################### values for checking
#S, K, r, drift, sigma, T
# 40, 40, 0.06, 0.06, 0.34641016151, 1
# option price should be around 2.33
# test price: 1.595757 ???
####################################################################################

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

1.333282
[-0.7380241]


# Change to CF Array

In [14]:
# change from CF matrix to CF array
# call
# r is constant here for testing

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  print('out')
  print(out)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)
  print('out_avg')
  print(out_avg)

  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  #print(ITM_matrix)
  CF_array = jnp.maximum(out_avg[:,-1]-K, 0) # array that will store discounted cash flows for the next time period
  print('CF_array')
  print(CF_array)

  # update time T
  # ECV_array = jax.ops.index_update(x=ECV_array, 
  #                                  idx=jax.ops.index[:,-1], 
  #                                  y=jnp.maximum(out_avg[:,-1]-K, 0)) # jnp array is immutable, so we'll need to reassign it every time we update. This is simply x[idx]=y
  
  # print(jax.numpy.linalg.lstsq(out_avg[jnp.array(jnp.where(ITM_matrix[:,-2])[0]),-2], 
  #                              y_sub))
  # ITM_index = jnp.where(ITM_matrix[:,-3])[0] # ITM index for this timestep
  # print(out_avg[ITM_index,-3])
  # X = out_avg[ITM_index,-3]
  # print(CF_matrix[ITM_index,-3:]) # subset of CF matrix, ITM, from timestep i to end
  # print(jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_matrix[:,-3].sum(), axis=0)) # discount matrix, same structure as the subset of CF matrix above
  # print(jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # take exponential
  # print(CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)) # multiply CF and discount matrices
  # print((CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)) # Y
  # Y = (CF_matrix[ITM_index,-3:] * jnp.exp(-jnp.repeat(jnp.array([r[0]*jnp.arange(3)]), ITM_index.shape[0], axis=0) * dt)).sum(axis=1)
  # print(jnp.array([X**0,X**1,X**2]).transpose()) # matrix of [X**0, X**1, X**2]
  # X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
  # print(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # regression coefficients
  # print(X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])) # predicted Y - ECV - expected continue value
  # ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0])
  # print(X-K)
  # print(X-K > ECV) # test if S-K > ECV
  # print((X-K > ECV).astype(int) * (X-K)) # X-K if X-K > ECV, else 0

  print('ITM_matrix')
  print(ITM_matrix)
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    print(i)
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    print('X')
    print(X)
    CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
    Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
    print('Y')
    print(Y)
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value
    print('ECV')
    print(ECV)

    print(jnp.array(ITM_index[jnp.where(X-K > ECV)[0]]))
    # update CF_array
    CF_array = jax.ops.index_update(x=CF_array, 
                                    idx=jax.ops.index[jnp.array(ITM_index[jnp.where(X-K > ECV)[0]])],  # index subsetting paths that has X-K > ECV
                                    y=(X-K)[jnp.where(X-K > ECV)[0]]) # X-K if X-K > ECV, else 0
    print('CF_array')
    print(CF_array)

  return  (CF_array * np.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 5

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 6
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 80.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

# delta test
# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths)) # here numsteps different from T

out
[[[ 96.21578 ]
  [ 91.14775 ]
  [100.194565]
  [ 98.64239 ]
  [100.26974 ]
  [110.80411 ]]

 [[ 76.17836 ]
  [ 88.16234 ]
  [ 89.02843 ]
  [ 88.396324]
  [ 76.27098 ]
  [ 62.58006 ]]

 [[ 96.975716]
  [ 85.91724 ]
  [ 84.55148 ]
  [ 88.01227 ]
  [ 78.10432 ]
  [ 75.73002 ]]

 [[100.92803 ]
  [105.0573  ]
  [116.7805  ]
  [146.18767 ]
  [152.94846 ]
  [128.80319 ]]

 [[ 94.14551 ]
  [ 88.50854 ]
  [ 73.177025]
  [ 71.277466]
  [ 65.9051  ]
  [ 75.834854]]]
out_avg
[[ 96.21578   91.14775  100.194565  98.64239  100.26974  110.80411 ]
 [ 76.17836   88.16234   89.02843   88.396324  76.27098   62.58006 ]
 [ 96.975716  85.91724   84.55148   88.01227   78.10432   75.73002 ]
 [100.92803  105.0573   116.7805   146.18767  152.94846  128.80319 ]
 [ 94.14551   88.50854   73.177025  71.277466  65.9051    75.834854]]
CF_array
[30.804108  0.        0.       48.803192  0.      ]
ITM_matrix
[[ True  True  True  True  True  True]
 [False  True  True  True False False]
 [ True  True  True  True False 

In [1]:
# change from CF matrix to CF array
# call
# r is constant here for testing
# large numpaths & numsteps to test price here

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg >= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(out_avg[:,-1]-K, 0) # array that will store discounted cash flows for the next time period
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
    Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value

    # update CF_array
    CF_array = jax.ops.index_update(x=CF_array, 
                                    idx=jax.ops.index[jnp.array(ITM_index[jnp.where(X-K > ECV)[0]])],  # index subsetting paths that has X-K > ECV
                                    y=(X-K)[jnp.where(X-K > ECV)[0]]) # X-K for these paths

  return  (CF_array * np.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 1000
numsteps = 100

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 90.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# delta test
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

14.875802
[0.75861514]


In [2]:
# change from CF matrix to CF array
# put
# r is constant here for testing
# large numpaths & numsteps to test price here

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period
 
  # update t=T-1 to t=0
  for i in range(2, numsteps+1):
    ITM_index = jnp.where(ITM_matrix[:,-i])[0]
    X = out_avg[ITM_index,-i]
    CF_array = (CF_array * np.exp(-r * dt)) # discount CF_array by one step
    Y = CF_array[ITM_index] # Y is subset of CF_array that is ITM
    X_matrix = jnp.array([X**0,X**1,X**2]).transpose()
    ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y)[0]) # predicted Y - ECV - expected continue value

    # update CF_array
    CF_array = jax.ops.index_update(x=CF_array, 
                                    idx=jax.ops.index[jnp.array(ITM_index[jnp.where(K-X > ECV)[0]])],  # index subsetting paths that has K-X > ECV
                                    y=(K-X)[jnp.where(K-X > ECV)[0]]) # K-X for these paths

  return  (CF_array * np.exp(-r * dt)).mean() # take mean for all paths to get price

numstocks = 1
numpaths = 1000
numsteps = 100

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
drift = jnp.array([0.01]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 110.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, numpaths)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# option price
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# delta test
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

11.943801
[-0.92381334]


# Construct Neural Net

In [ ]:
%%writefile cupy_dataset.py
import cupy
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, keys): # need to pass 'keys'
    return jnp.mean((jnp.maximum(jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T)[:,-1,:], axis=1)-K,0)) * jnp.exp(-r[0] * T))
    # must use '-1' not 'numsteps', or else grad will be 0

###################################################################################################
# these 2 functions must be defined outside class in order to be used in 'optionvalueavg' function
fast_simple = jax.jit(Brownian_motion, static_argnums=2)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))
###################################################################################################

class OptionDataSet(object):
    
    def __init__(self, max_len, number_path, batch, seed, stocks):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 50
        self.N_BATCH = batch
        self.N_STOCKS = stocks
        self.T = 1.0 # assume T = 1, use float here
        self.seed = seed
        np.random.seed(seed)
        
    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num >= self.max_length:
            raise StopIteration
        
        Y = cupy.zeros((self.N_BATCH, 1 + self.N_STOCKS), dtype=cupy.float32) # output: price, delta1, delta2, delta3
        X = cupy.zeros((self.N_BATCH, self.N_STOCKS * 6), dtype = cupy.float32)

        for op in range(self.N_BATCH):
          
          rng = jax.random.PRNGKey(self.seed)
          rng, key = jax.random.split(rng)

          ################################################################################################### generate random input numbers

          initial_stocks = jnp.array(np.random.random(self.N_STOCKS) * 200.0)

          corr = jnp.diag(jnp.array([1]*self.N_STOCKS)) # assume no correlation between stocks here
          sigma = jnp.array(np.random.random(self.N_STOCKS) * 0.4)
          cov = (jnp.diag(sigma)).dot(corr).dot(jnp.diag(sigma))

          r = jnp.repeat(jnp.array(np.random.random(1) * 0.1), self.N_STOCKS)
          drift = r # To match BS, use drift = r

          T = self.T
          K = np.random.random(1) * 200.0

          ###################################################################################################
          ################################################################################################### apply functions to compute price and deltas
          
          keys = jax.random.split(key, self.N_PATHS)

          European_Call_price = optionvalueavg(key, initial_stocks, self.N_STEPS, drift, r, cov, K, T, keys) # need to pass 'keys'
          gooptionvalue = jax.grad(optionvalueavg, argnums=1)
          Deltas = gooptionvalue(keys, initial_stocks, self.N_STEPS, drift, r, cov, K, T, keys) # need to pass 'keys'

          ###################################################################################################
          ################################################################################################### store input and output numbers in X and Y

          Y[op, 0] = European_Call_price
          Y[op, 1:4] = cupy.array(Deltas, dtype=cupy.float32)

          paras = (jnp.repeat(jnp.array(T), self.N_STOCKS), jnp.repeat(jnp.array(K), self.N_STOCKS), initial_stocks, sigma, drift, r)
          paras = np.column_stack(paras).reshape(1,-1)[0]
          X[op,] = cupy.array(paras)

          ###################################################################################################

        self.num += 1
        return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))


# ds = OptionDataSet(max_len = 2, number_path = 10000, batch = 2, seed = 15, stocks=3) # for testing purpose, use constant seed. When training, change to random seed
# for i in ds:
#     print(i)

Writing cupy_dataset.py


In [ ]:
%%writefile model.py
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np

class Net(nn.Module):

    def __init__(self, hidden=1024):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6*3, hidden) # remember to change this!
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, hidden)
        self.fc5 = nn.Linear(hidden, hidden)
        self.fc6 = nn.Linear(hidden, 4) # 4 outputs: price, delta1, delta2, delta3
        self.register_buffer('norm',
                             torch.tensor([1, 200.0, 200.0, 0.4, 0.1, 0.1]*3)) # don't use numpy here - will give error later

    def forward(self, x):
        # normalize the parameter to range [0-1] 
        x = x / self.norm
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = F.elu(self.fc4(x))
        x = F.elu(self.fc5(x))
        return self.fc6(x)

Writing model.py


# Train Neural Net

In [ ]:
!pip install pytorch-ignite

     |████████████████████████████████| 232 kB 10.0 MB/s 


In [ ]:
# If memory is not enough, try changing parameters and restarting session
# loss will converge

from ignite.engine import Engine, Events
from ignite.handlers import Timer
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import OptionDataSet
import numpy as np
timer = Timer(average=True)
model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
dataset = OptionDataSet(max_len = 100, number_path = 1024, batch = 32, seed = np.random.randint(10000), stocks = 3) # must have random seed


def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    #print(y)
    y_pred = model(x)
    #print(y_pred)
    loss = loss_fn(y_pred[:,:], y[:,:]) # compute MSE between the 2 arrays
    #print(loss)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 20

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value(), 'iter num', iter)
        
trainer.run(dataset, max_epochs = 100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: /usr/local/lib/python3.7/dist-packages/ignite/contrib/handlers/param_scheduler.py has been moved to /ignite/handlers/param_scheduler.py and will be removed in version 0.6.0.
 Please refer to the documentation for more details.
  """


loss 308.3535461425781 average time 0.5890158134499984 iter num 20
loss 309.395263671875 average time 0.29650749102499674 iter num 40
loss 319.1919250488281 average time 0.1990294852333278 iter num 60
loss 232.41702270507812 average time 0.15041632617499587 iter num 80
loss 169.25584411621094 average time 0.12112373191999723 iter num 100
loss 41.82872009277344 average time 0.0727944900500006 iter num 20
loss 8.219586372375488 average time 0.038342809124999634 iter num 40
loss 2.8115668296813965 average time 0.026894211649999042 iter num 60
loss 2.778562545776367 average time 0.021186888737497613 iter num 80
loss 2.3859236240386963 average time 0.017774565589996884 iter num 100
loss 1.3037325143814087 average time 0.07461170490000768 iter num 20
loss 1.4311182498931885 average time 0.039529715224995245 iter num 40
loss 0.608491837978363 average time 0.027656891716664706 iter num 60
loss 0.5415693521499634 average time 0.02177082573749658 iter num 80
loss 0.3868729770183563 average time 

ERROR:ignite.engine.engine.Engine:Engine run is terminating due to exception: 


KeyboardInterrupt: ignored

**Save Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
model_save_name = 'jax_european_test_2.pth'
path = F"/content/drive/MyDrive/AFP Project/PUI/{model_save_name}" 
torch.save(model.state_dict(), path)

**Load Model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
model_save_name = 'Sobolev_test_2.pth'
path = F"/content/drive/MyDrive/AFP Project/PUI/{model_save_name}" 
state_dict = torch.load(path)
print(state_dict.keys())

odict_keys(['norm', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias', 'fc5.weight', 'fc5.bias', 'fc6.weight', 'fc6.bias'])


In [ ]:
# need to run 'Writing cupy_dataset.py' and 'Writing model.py' above before this
from model import Net
model = Net().cuda()

model.load_state_dict(state_dict)
print(model)

Net(
  (fc1): Linear(in_features=18, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=1024, bias=True)
  (fc5): Linear(in_features=1024, out_features=1024, bias=True)
  (fc6): Linear(in_features=1024, out_features=7, bias=True)
)


**Continue to train model**

In [ ]:
# If memory is not enough, try changing parameters and restarting session
# loss will converge

from ignite.engine import Engine, Events
from ignite.handlers import Timer
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import OptionDataSet
import numpy as np
timer = Timer(average=True)
#model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
#dataset = OptionDataSet(max_len = 100, number_path = 1024, batch = 32, seed = np.random.randint(10000), stocks = 3) # must have random seed
dataset = OptionDataSet(max_len = 100, number_path = 10000, batch = 8, seed = np.random.randint(10000), stocks = 3) # must have random seed


def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    #print(y)
    y_pred = model(x)
    #print(y_pred)
    loss = loss_fn(y_pred[:,:], y[:,:]) # compute MSE between the 2 arrays
    #print(loss)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 20

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value(), 'iter num', iter)
        
trainer.run(dataset, max_epochs = 10)

model_save_name = 'jax_european_test_3.pth'
path = F"/content/drive/MyDrive/AFP Project/PUI/{model_save_name}" 
torch.save(model.state_dict(), path)

loss 3.847933769226074 average time 0.024525942649995615 iter num 20
loss 0.11467143893241882 average time 0.014268842749987697 iter num 40
loss 0.7498358488082886 average time 0.010797686849988017 iter num 60
loss 0.5012774467468262 average time 0.00905447164998634 iter num 80
loss 0.08808335661888123 average time 0.008048051719988507 iter num 100
loss 0.5424646139144897 average time 0.023962988050016065 iter num 20
loss 0.09199616312980652 average time 0.013907441924999375 iter num 40
loss 0.41971147060394287 average time 0.010556156083328005 iter num 60
loss 0.9499955773353577 average time 0.008943276199994443 iter num 80
loss 0.039512861520051956 average time 0.007942020249997768 iter num 100
loss 0.038578908890485764 average time 0.023953661050018126 iter num 20
loss 0.7367726564407349 average time 0.013887499250017754 iter num 40
loss 0.3312565088272095 average time 0.010584145183342268 iter num 60
loss 0.16353926062583923 average time 0.008966675425003245 iter num 80
loss 0.0240

#Results

In [ ]:
import torch
inputs = torch.tensor([[1, 110.0, 100.0, 0.25, 0., 0.]*3]).cuda()
model(inputs.float())

# price, delta1, delta2, delta3
# should be around (2.3654, 0.0937, 0.0937, 0.0937)

tensor([[2.3494, 0.0840, 0.0863, 0.0899]], device='cuda:0',
       grad_fn=<AddmmBackward>)